In [242]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import math

In [243]:
anime_dicts = pd.read_pickle("../data/clean_anime_dicts.pickle")
anime_sales_df = pd.read_pickle("../data/anime_sales_df_matched.pickle")

In [244]:
addl_anime_info = pd.DataFrame.from_dict(anime_dicts, orient="index")
anime_sales_df = pd.concat([anime_sales_df.loc[addl_anime_info.index], addl_anime_info], axis=1)


In [245]:
anime_sales_df["sales"] = anime_sales_df["gross_1st_rls"].map(lambda x: x[1:].replace(",", "")).astype(float)

In [246]:
anime_sales_df = anime_sales_df[anime_sales_df["sales"] != 0]
anime_sales_df = anime_sales_df[anime_sales_df["rating"] != "None"]
anime_sales_df = anime_sales_df[anime_sales_df["duration"] != "Unknown"]

anime_sales_df = anime_sales_df.dropna(subset=["score"])

In [247]:
anime_sales_df["broadcast"] = anime_sales_df["broadcast"].apply(lambda x: (None, None) if not x else x)

In [248]:
anime_sales_df["broadcast"] = anime_sales_df["broadcast"].apply(lambda x: (None, None) if not x else x)
anime_sales_df[["day", "time"]] = pd.DataFrame(anime_sales_df["broadcast"].tolist(), index=anime_sales_df.index)
anime_sales_df["hour"] = anime_sales_df["time"].apply(lambda x: int(x[:2]) if x else None)
anime_sales_df["duration"] = anime_sales_df["duration"].astype(int)
anime_sales_df["episode"] = anime_sales_df["episode"].astype(int)
anime_sales_df["runtime"] = anime_sales_df["episode"]*anime_sales_df["duration"]
anime_sales_df["year"] = anime_sales_df["year"].astype(int)

In [249]:
genre_list = np.unique(anime_sales_df["genre"].aggregate(sum))

# series of series, each series has columns of genres and 1 if it's of that genre
ser_ser = anime_sales_df["genre"].map(lambda x: pd.Series(1, index=x))
# turn series of series into a df with same indices as original df
genre_df = ser_ser.apply(lambda x: x)
genre_df = genre_df.fillna(0)

anime_sales_df = pd.concat([anime_sales_df, genre_df], axis=1)


In [250]:
column_list = anime_sales_df.columns.tolist()
column_list.remove("sales")
column_list.append("sales")

anime_sales_df = anime_sales_df[column_list]

In [251]:
anime_sales_df.to_pickle("../data/anime_sales_df_v3.pickle")

In [252]:
anime_sales_df.columns

Index(['title', 'year', 'season_qtr', 'avg_sales', 're_rls', 'total',
       'gross_1st_rls', 'studio', 'source', 'episode', 'broadcast', 'genre',
       'duration', 'rating', 'score', 'member', 'favorite', 'day', 'time',
       'hour', 'runtime', 'Game', 'Sci-Fi', 'Adventure', 'Mystery', 'Magic',
       'Fantasy', 'Comedy', 'Shounen', 'Drama', 'Action', 'Mecha', 'Seinen',
       'Demons', 'Josei', 'Military', 'Ecchi', 'Super Power', 'Supernatural',
       'Music', 'Romance', 'School', 'Slice of Life', 'Parody', 'Shoujo',
       'Historical', 'Harem', 'Police', 'Sports', 'Martial Arts',
       'Psychological', 'Shoujo Ai', 'Space', 'Samurai', 'Horror', 'Thriller',
       'Vampire', 'Shounen Ai', 'Dementia', 'Cars', 'sales'],
      dtype='object')